In [1]:
from langchain.llms import CTransformers  # to load the llama2 model

In [2]:
llm = CTransformers(model="model\llama-2-7b-chat.ggmlv3.q3_K_M.bin",
                        model_type="llama", config={'max_new_tokens':3900,'temperature': 0.2,'context_length': 3800})

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_community.utilities import SQLDatabase


# MySQL connection URI
username = 'root'
password = 'prabal9869'
host = '127.0.0.1'
dbname = 'arl_bank'  # Database name

# Constructing the MySQL URI
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}/{dbname}"

# # Initializing SQLDatabase object for MySQL
db = SQLDatabase.from_uri(mysql_uri, sample_rows_in_table_info=3)
print(db.table_info)


CREATE TABLE transactions (
	`Account_No` VARCHAR(50) NOT NULL, 
	`Transaction_details` TEXT, 
	`Withdrawal_amount` INTEGER, 
	`Deposit_amount` INTEGER, 
	`Balance_amount` INTEGER, 
	`Value_date` DATE, 
	`Date` DATE
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from transactions table:
Account_No	Transaction_details	Withdrawal_amount	Deposit_amount	Balance_amount	Value_date	Date
409000611074'	TRF FROM  Indiaforensic SERVICES	0	1000000	1000000	2022-10-05	2022-10-05
409000611074'	TRF FROM  Indiaforensic SERVICES	0	1000000	2000000	2022-10-11	2022-10-11
409000611074'	FDRL/INTERNAL FUND TRANSFE	0	500000	2500000	2022-10-24	2022-10-24
*/


In [4]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm=llm,db=db,verbose=True)


# FEW SHORT LEARNING

In [5]:
few_shots=[
    {
        'Question':" Income of last 3 months.My Account Number is 409000493201'. ?",
        'SQLQuery':"""SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Account_No = "409000493201'" AND Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH);
        """,
        'SQLResult':"16725509",
        'Answer':"16725509 is the income of last 3 months."
    },

    {
        'Question':"Total expenses of last 8 months for account number 409000611074' ?",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE 
Account_No = "409000611074'" AND Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 8 MONTH);
        """,
        'SQLResult':"75124046",
        'Answer':"75124046 is the total expenses of last 8 months."
    },
      {
        'Question':"Total saving of last month as my account number is 409000493201'  ?",
        'SQLQuery':"""SELECT (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Savings_Last_Month FROM 
transactions WHERE Account_No = "409000493201'" AND YEAR(Value_date) = YEAR(CURRENT_DATE() 
- INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH);
        """,
        'SQLResult':"-193509",
        'Answer':"You saved -193509 last month."
    },
    {
        'Question':"How many transactions did I make in last 2 week as my account number is 409000493201' ",
        'SQLQuery':"""SELECT COUNT(*) AS Total_Transactions FROM transactions WHERE 
Account_No = "409000493201'" AND Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 WEEK);
        """,
        'SQLResult':"24",
        'Answer':"You made 24 transactions in last 2 week."
    },
     {
        'Question':"Amount spent last week as my account number is 409000493201' ",
        'SQLQuery':"""SELECT SUM(Withdrawal_amount) AS Total_Spending FROM transactions WHERE Account_No = "409000493201'" AND 
Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 WEEK);
        """,
        'SQLResult':"515559",
        'Answer':"515559 is your total spending last week."
    }
    , {
        'Question': "Expenses for each day this month as my account number is  409000493201' ?",
        'SQLQuery': """SELECT Date, SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE Account_No = "409000493201'" AND YEAR(Value_date) = YEAR(CURRENT_DATE()) AND MONTH(Value_date) = MONTH(CURRENT_DATE()) GROUP BY Date;
""",
        'SQLResult':
        """
Date                        Total_Expenses
2024-06-02                   183648
2024-06-03                   80666
2024-06-04                   135504
2024-06-05                   92031
2024-06-06                   118475
2024-06-07                   16440
2024-06-09                   62112
2024-06-10                   10331
"""
        ,
        'Answer': """The total_expenses is 183648,80666,135504,92031,118475,16440,62112,10331 for the year 2024 month of 6 and date 02,03,04,05,06,07,09 and 10.  """         
    },
    {
        'Question':"savings of previous year for each month as my account number is 409000493201' ",
        'SQLQuery':"""SELECT MONTH(Value_date) AS Month, (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Total_Savings 
FROM transactions WHERE Account_No = "409000493201'" AND YEAR(Value_date) = YEAR(CURRENT_DATE() - INTERVAL 1 YEAR) GROUP BY MONTH(Value_date) ORDER BY MONTH(Value_date);
        """,
        'SQLResult':"""
        Date                Total_Expenses
        1                       -311244
        2                          9101
        3                         3476          
        4                       -15971
        5                       322249
        6                       -172560
        7                       361471
        8                       -446844
        9                       93215
        10                      -73917
        11                      294502
        12                      -404028""",
        'Answer':"Month-1:-311244, Month-2:9101, Month-3:3476"
    },
     {
        'Question':"Amount Saved each year as my account number is  409000493201'",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year,SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Savings
                FROM transactions WHERE Account_No = "409000493201'" GROUP BY YEAR(Value_date);
        """,
        'SQLResult':"""
        Year                Savings
        2021                 497766
        2022                 272458 
        2023                 -340550          
        2024                 -148114
       """,
        'Answer':"You saved 497766 in 2021 ,272458 in 2022, -340550 in 2023 and -148114 in 2024."
    }
    ,
     {
        'Question':"Savings of last 2 year as my account number is 409000611074' ",
        'SQLQuery':"""SELECT YEAR(Value_date) AS Year, SUM(Deposit_amount) - SUM(Withdrawal_amount) AS Net_Savings FROM transactions WHERE Account_No = "409000611074'" AND Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 YEAR) GROUP BY YEAR(Value_date);
        """,
        'SQLResult':"""
        Year                Savings
        2022                 2270453
        2023                 -895783          
        2024                 -912470
        """,
        'Answer':"You saved 2270453 in 2022, -89578 in 2023 and -912470 in 2024."
    }
    ,
     {
        'Question':"Expenses of each day of last week for account number 409000493201' ",
        'SQLQuery':"""SELECT DATE(Value_date) AS Date, SUM(Withdrawal_amount) AS Total_Spending 
FROM transactions WHERE Account_No = '409000493201'AND Value_date >= DATE_SUB(CURDATE(), INTERVAL 1 WEEK) GROUP BY DATE(Value_date);

        """,
        'SQLResult':"""
        Year                Savings
        2024-06-06           118475
        2024-06-07           16440         
        2024-06-09           62112
        2024-06-10           10331

        """,
        'Answer':"You spend 118475,16440,62112,10331."
    },
    {
        'Question':"Amount spent this year as my account number is 409000493201' ?",
        'SQLQuery':"""SELECT year(Value_date) as Year,sum(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE 
Account_No = "409000493201'" AND year(Value_date) = year(curdate())
group by year(Value_date);
        """,
        'SQLResult':"""
        Year                Total_Expenses
       2024                    36616174

        """,
        'Answer':"You spent 36616174 this year."
    }
    
]



In [6]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import json


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

to_vectorize = [" ".join(example.values()) for example in few_shots]

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
to_vectorize 

[' Income of last 3 months.My Account Number is 409000493201\'. ? SELECT SUM(Deposit_amount) AS Total_Income FROM transactions WHERE Account_No = "409000493201\'" AND Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 3 MONTH);\n         16725509 16725509 is the income of last 3 months.',
 'Total expenses of last 8 months for account number 409000611074\' ? SELECT SUM(Withdrawal_amount) AS Total_Expenses FROM transactions WHERE \nAccount_No = "409000611074\'" AND Value_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 8 MONTH);\n         75124046 75124046 is the total expenses of last 8 months.',
 'Total saving of last month as my account number is 409000493201\'  ? SELECT (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Savings_Last_Month FROM \ntransactions WHERE Account_No = "409000493201\'" AND YEAR(Value_date) = YEAR(CURRENT_DATE() \n- INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH);\n         -193509 You saved -193509 last month.',
 'How many transactions

In [8]:

vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [9]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=1,
)
example_selector.select_examples({"Question": "How much amount did i save this year as my account number is 409000493201'  "})

[{'Answer': 'You saved -193509 last month.',
  'Question': "Total saving of last month as my account number is 409000493201'  ?",
  'SQLQuery': 'SELECT (SUM(Deposit_amount) - SUM(Withdrawal_amount)) AS Savings_Last_Month FROM \ntransactions WHERE Account_No = "409000493201\'" AND YEAR(Value_date) = YEAR(CURRENT_DATE() \n- INTERVAL 1 MONTH) AND MONTH(Value_date) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH);\n        ',
  'SQLResult': '-193509'}]

In [10]:
# Define the custom prompt for SQL database interactions
custom_mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question. Prepare the correct MySQL query by looking at the top {top_k} similar queries.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today","this year","this month" ,"this week".

Table information:
1. `transactions` with columns: `Account_No`, `Date`, `Transaction_details`, `Value_date`, `Withdrawal_amount`, `Deposit_amount`, `Balance_amount`

Remember to only use the provided table columns and structure your queries to handle the specified requests accurately.Only answer the question given by the user and dont assume any question by yourself.
 """

In [11]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX

In [12]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [13]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [14]:
suffix_eg="Question: {input}"

In [15]:
print(suffix_eg)

Question: {input}


In [16]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=custom_mysql_prompt,
    suffix=suffix_eg,
    input_variables=["input", "top_k"], 
)

In [17]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)


In [18]:
print(few_shot_prompt)

input_variables=['input', 'top_k'] example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001A008137290>, k=1, example_keys=None, input_keys=None, vectorstore_kwargs=None) example_prompt=PromptTemplate(input_variables=['Answer', 'Question', 'SQLQuery', 'SQLResult'], template='\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}') suffix='Question: {input}' prefix='You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question. Prepare the correct MySQL query by looking at the top {top_k} similar queries.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.\nNever query for all colum

In [19]:
# result = new_chain("What was my total earning last week as my account number is 409000493201'")

In [20]:
example_questions = [shot['Question'].strip() for shot in few_shots]
print(example_questions)

["Income of last 3 months.My Account Number is 409000493201'. ?", "Total expenses of last 8 months for account number 409000611074' ?", "Total saving of last month as my account number is 409000493201'  ?", "How many transactions did I make in last 2 week as my account number is 409000493201'", "Amount spent last week as my account number is 409000493201'", "Expenses for each day this month as my account number is  409000493201' ?", "savings of previous year for each month as my account number is 409000493201'", "Amount Saved each year as my account number is  409000493201'", "Savings of last 2 year as my account number is 409000611074'", "Expenses of each day of last week for account number 409000493201'", "Amount spent this year as my account number is 409000493201' ?"]


In [21]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
example_embeddings = model.encode(example_questions)

In [64]:
from sklearn.metrics.pairwise import cosine_similarity  
# Function to determine if a user question is relevant to the database context
def is_question_relevant(user_question, similarity_threshold=0.76):
    user_embedding = model.encode([user_question])[0]  
    
    # calculating the cosine similarity between user question and example questions
    similarity_scores = cosine_similarity(user_embedding.reshape(1, -1), example_embeddings)[0]
    print(similarity_scores)
    
    max_similarity_score = max(similarity_scores)
    print(max_similarity_score)
    return max_similarity_score > similarity_threshold

In [65]:
from guardrails import Guard, OnFailAction
from guardrails.hub import CompetitorCheck, ToxicLanguage
from guardrails.hub import ProfanityFree

In [66]:


guard = Guard().use_many(
    CompetitorCheck(["khalti", "fusemachine"], on_fail=OnFailAction.EXCEPTION),
    ToxicLanguage(threshold=0.9, validation_method="sentence", on_fail=OnFailAction.EXCEPTION),
    ProfanityFree(on_fail=OnFailAction.EXCEPTION))



c:\Users\Prabal Kuinkel\Desktop\ARL-Intern\privategpt\privategpt\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [67]:
# Function to handle user queries
def handle_user_query(user_question):
    if is_question_relevant(user_question):
        # response=new_chain(user_question)
        return True
    else:
        return "Your Question is not Relevant"

In [85]:
user_question = "what khalti is as my account number is 409000493201' "

In [86]:
try:
    # Validate the user question
    guard.validate(user_question)  # If validation fails, an exception will be thrown
    # If validation passes, handle the user query
    response = handle_user_query(user_question)
except Exception as e:
    response = str(e)

In [87]:
print(response)

Validation failed for field with errors: Found the following competitors: [['khalti']]. Please avoid naming those competitors next time


In [88]:
if response==True:
    new_chain(user_question)
else:
    print(response)

Validation failed for field with errors: Found the following competitors: [['khalti']]. Please avoid naming those competitors next time


In [31]:
# db_chain.run("""SELECT SUM(Deposit_amount) - SUM(Withdrawal_amount) 
#              AS Savings FROM transactions WHERE Account_No = "409000493201'" AND year(Value_date) = year(curdate())""")